# Process terms 
Process terms retrieved from the neural network model using `LIME`

---

## Load the retrieved terms

In [1]:
import os

In [2]:
root_dir = '../..'
data_dir = 'data/terms'
filename = 'relevant_terms_alaska_nn.csv'
filepath = os.path.join(root_dir, data_dir, filename)

In [3]:
import pandas as pd

In [4]:
terms_df = pd.read_csv(filepath)

In [5]:
terms_df.head()

,label,term,weight,data_id
0,ENTITY#44,nikon_d3200_dslr_camera,0.563572,0
1,ENTITY#44,18_55mm,0.265820,0
2,ENTITY#44,55_200mm_lenses,0.175857,0
3,ENTITY#44,black_ebay,0.129290,0
4,ENTITY#44,nikon_d3200,0.812544,1


---

## Rank terms

### Split noun chunks into single terms

In [6]:
def split_noun_chunks(input_df):
    df_data = []
    
    for _, row in input_df.iterrows():
        noun_chunk = row['term']

        for term in noun_chunk.split('_'):
            data_dict = {'label': row['label'], 
                         'term': term, 
                         'weight': row['weight'], 
                         'data_id': row['data_id']}
            df_data.append(data_dict)
    
    return pd.DataFrame(df_data)

In [7]:
terms_df = split_noun_chunks(terms_df)

In [8]:
terms_df.head(10)

,label,term,weight,data_id
0,ENTITY#44,nikon,0.563572,0
1,ENTITY#44,d3200,0.563572,0
2,ENTITY#44,dslr,0.563572,0
3,ENTITY#44,camera,0.563572,0
4,ENTITY#44,18,0.265820,0
5,ENTITY#44,55mm,0.265820,0
6,ENTITY#44,55,0.175857,0
7,ENTITY#44,200mm,0.175857,0
8,ENTITY#44,lenses,0.175857,0
9,ENTITY#44,black,0.129290,0


### Aggregate (sum) LIME weights for each term

In [9]:
terms_stats_df = terms_df.groupby(['label', 'term']).agg({'weight': 'sum'}).reset_index()

In [10]:
terms_stats_df.head()

,label,term,weight
0,ENTITY#101,16-35mm,0.724391
1,ENTITY#101,17-40mm,0.676306
2,ENTITY#101,2,0.055329
3,ENTITY#101,22,5.230077
4,ENTITY#101,24-105mm,0.724367


### Sort terms
Rank terms by total weight (descending)

In [11]:
ranking_df = terms_stats_df.groupby('label')\
                .apply(lambda grp: grp.sort_values(by='weight', ascending=False))\
                .reset_index(drop=True)

In [12]:
ranking_df.groupby('label').head(3)

,label,term,weight
0,ENTITY#101,5d,96.933690
1,ENTITY#101,mark,93.036158
2,ENTITY#101,iii,88.471847
50,ENTITY#102,5d,63.397395
51,ENTITY#102,canon,61.651098
52,ENTITY#102,eos,55.676296
101,ENTITY#16,d90,83.321507
102,ENTITY#16,nikon,82.346056
103,ENTITY#16,camera,20.047362
163,ENTITY#18,canon,120.404014


---

## Save ranking

In [13]:
ranking_filename = 'ranking_alaska_nn_single_terms.xlsx'
ranking_filepath = os.path.join(root_dir, data_dir, ranking_filename)

In [14]:
ranking_filepath

'../../data/terms/ranking_alaska_nn_single_terms.xlsx'

In [15]:
ranking_df.to_excel(ranking_filepath)

---